In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image
import numpy as np

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
x_train.shape

(60000, 28, 28)

In [4]:
y_train.shape

(60000,)

In [5]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

In [6]:
batch_size = 128
num_classes = 10
epochs = 25
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [7]:
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [8]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])

In [9]:
hist = model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,
                 verbose=1,validation_data=(x_test, y_test))
print("The model has successfully trained")

Epoch 1/25
469/469 [==============================] - 88s 186ms/step - loss: 26.7939 - accuracy: 0.1979 - val_loss: 3.0927 - val_accuracy: 0.6227
Epoch 2/25
469/469 [==============================] - 87s 185ms/step - loss: 12.6950 - accuracy: 0.3672 - val_loss: 1.7507 - val_accuracy: 0.7521
Epoch 3/25
469/469 [==============================] - 86s 183ms/step - loss: 7.5090 - accuracy: 0.4685 - val_loss: 1.1554 - val_accuracy: 0.7953
Epoch 4/25
469/469 [==============================] - 88s 187ms/step - loss: 5.0476 - accuracy: 0.5273 - val_loss: 0.8226 - val_accuracy: 0.8125
Epoch 5/25
469/469 [==============================] - 88s 188ms/step - loss: 3.6890 - accuracy: 0.5596 - val_loss: 0.6538 - val_accuracy: 0.8173
Epoch 6/25
469/469 [==============================] - 89s 189ms/step - loss: 2.8384 - accuracy: 0.5755 - val_loss: 0.6163 - val_accuracy: 0.8139
Epoch 7/25
469/469 [==============================] - 88s 187ms/step - loss: 2.3473 - accuracy: 0.5803 - val_loss: 0.6671 - val_

In [10]:
model.save('mnist.h5')
print("Saving the model as mnist.h5")

Saving the model as mnist.h5


In [11]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.4407407343387604
Test accuracy: 0.8938999772071838


In [ ]:
from keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image
import numpy as np

model = load_model('mnist.h5')

def predict_digit(img):
    #resize image to 28x28 pixels
    img = img.resize((28,28))
    #convert rgb to grayscale
    img = img.convert('L')
    img = np.array(img)
    #reshaping to support our model input and normalizing
    img = img.reshape(1,28,28,1)
    img = img/255.0
    #predicting the class
    res = model.predict([img])[0]
    return np.argmax(res), max(res)

class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)

        self.x = self.y = 0

        # Creating elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg = "white", cursor="cross")
        self.label = tk.Label(self, text="Thinking..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text = "Recognise", command =         self.classify_handwriting) 
        self.button_clear = tk.Button(self, text = "Clear", command = self.clear_all)

        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=W, )
        self.label.grid(row=0, column=1,pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)

        #self.canvas.bind("<Motion>", self.start_pos)
        self.canvas.bind("<B1-Motion>", self.draw_lines)

    def clear_all(self):
        self.canvas.delete("all")

    def classify_handwriting(self):
        HWND = self.canvas.winfo_id() # get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND) # get the coordinate of the canvas
        im = ImageGrab.grab(rect)

        digit, acc = predict_digit(im)
        self.label.configure(text= str(digit)+', '+ str(int(acc*100))+'%')

    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r=8
        self.canvas.create_oval(self.x-r, self.y-r, self.x + r, self.y + r, fill='black')

app = App()
mainloop()

1/1 [==============================] - 0s 79ms/step
